<a href="https://colab.research.google.com/github/FGC97/project1_Henry/blob/main/project1_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing libraries
import pandas as pd
import ast
import math
import numpy as np

In [ ]:
# Drive
# from google.colab import drive
# drive.mount('/content/drive')

# credits = pd.read_csv('/content/drive/MyDrive/Proyectos Henry/ML_Ops_Project/credits.csv', sep = ',')
# movies = pd.read_csv('/content/drive/MyDrive/Proyectos Henry/ML_Ops_Project/movies_dataset.csv', sep = ',')

In [ ]:
# Reading the 'credits.csv' file and storing the data in the 'credits' DataFrame
credits = pd.read_csv('dataset/credits.csv', sep=',')

# # Reading the 'movies_dataset.csv' file and storing the data in the 'movies' DataFrame
movies = pd.read_csv('dataset/movies_dataset.csv', sep=',')

In [ ]:
print(credits.info())
print(movies.info())

# **Unpacking nested lists of dictionaries**




# Data Frame ***credits.csv***


## Column Crew: Extracting Directors
To extract the directors from the crew list, we can define a function called get_directors that iterates over each dictionary in the crew list and returns a list of director names.

In [ ]:
# Verifying the data of the elements of "crew" and "cast". They are a list of dictionaries in string format.

print(type(credits['crew'][0]))
print(type(credits['cast'][0]))

<class 'str'>
<class 'str'>


In [ ]:
# Verifying the value of the second row
type(credits['crew'][1])

# Applying the ast.literal_eval function to convert the values in the 'crew' column
# from their string representation to a list of dictionaries

credits['crew'] = credits['crew'].apply(ast.literal_eval)

In [ ]:
# Creating a function to iterate through each dictionary in the "crew" list and return a list of directors for each movie:
def get_directors(crew):
    directors = []
    for member in crew:
        if member["job"] == "Director":
            directors.append(member["name"])
    return directors

# Creating a new column in the "credits" dataframe by applying the "get_directors" function to each record in the "crew" column:
credits['director'] = credits['crew'].apply(get_directors)



In [ ]:
credits.head(3)

,cast,crew,id,director
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,[John Lasseter]
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,[Joe Johnston]
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,[Howard Deutch]


## Column Cast: Extracting Actors

To extract the actors from the cast list, we can define a function called get_actors that iterates over each dictionary in the cast list and returns a list of actor names.

In [ ]:
# Verifying the value of the second row for cast
type(credits['cast'][1])

# Applying ast.literal_eval to convert 'cast' column to a list
credits['cast'] = credits['cast'].apply(ast.literal_eval)

In [ ]:
# Function to extract actors from 'cast' list
def get_actors(cast):
    actor_names = []
    for actor in cast:
        if 'name' in actor:
            actor_names.append(actor['name'])
    return actor_names

# Applying the 'get_actors' function to create a new column 'actors' in the 'credits' DataFrame
credits['actors'] = credits['cast'].apply(get_actors)


In [ ]:
print(credits.head(3))

                                                cast  \
0  [{'cast_id': 14, 'character': 'Woody (voice)',...   
1  [{'cast_id': 1, 'character': 'Alan Parrish', '...   
2  [{'cast_id': 2, 'character': 'Max Goldman', 'c...   

                                                crew     id         director  \
0  [{'credit_id': '52fe4284c3a36847f8024f49', 'de...    862  [John Lasseter]   
1  [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...   8844   [Joe Johnston]   
2  [{'credit_id': '52fe466a9251416c75077a89', 'de...  15602  [Howard Deutch]   

                                              actors  
0  [Tom Hanks, Tim Allen, Don Rickles, Jim Varney...  
1  [Robin Williams, Jonathan Hyde, Kirsten Dunst,...  
2  [Walter Matthau, Jack Lemmon, Ann-Margret, Sop...  


## Cleaning Credits Data Frame

In [ ]:
# Remove the 'cast' and 'crew' columns from the DataFrame called credits.
credits = credits.drop(['cast', 'crew'], axis=1)

print(credits.head(3))
print(credits.info())


# Checks for null values
print(credits.isnull().sum())

      id         director                                             actors
0    862  [John Lasseter]  [Tom Hanks, Tim Allen, Don Rickles, Jim Varney...
1   8844   [Joe Johnston]  [Robin Williams, Jonathan Hyde, Kirsten Dunst,...
2  15602  [Howard Deutch]  [Walter Matthau, Jack Lemmon, Ann-Margret, Sop...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        45476 non-null  int64 
 1   director  45476 non-null  object
 2   actors    45476 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.0+ MB
None
id          0
director    0
actors      0
dtype: int64


# Data Frame ***movies.csv***

In [ ]:
# Function to convert a string to a list, handling null values.

def convert_to_dict(value):
    if pd.notnull(value):
        try:
            return ast.literal_eval(value)
        except (ValueError, SyntaxError):
            return None
    else:
        return None

## **belongs_to_collection** column

In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [ ]:
# Verifying the data of the elements of "belongs_to_collection". They are a list of dictionaries in string format.
print(movies['belongs_to_collection'][0])
type(movies['belongs_to_collection'][0])

{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}


str

In [ ]:
movies['belongs_to_collection'].head(5)

0    {'id': 10194, 'name': 'Toy Story Collection', ...
1                                                  NaN
2    {'id': 119050, 'name': 'Grumpy Old Men Collect...
3                                                  NaN
4    {'id': 96871, 'name': 'Father of the Bride Col...
Name: belongs_to_collection, dtype: object

In [ ]:
# Applying the function to convert from string to list
movies['belongs_to_collection'] = movies['belongs_to_collection'].apply(convert_to_dict)

In [ ]:
# I want to extract the names of the collections but it fails for 3 Float data
# collection_names = movies['belongs_to_collection'].apply(lambda x: x['name'] if x is not None else None)
# collection_names


# I check where those float values are, by their index
for index, row in movies.iterrows():
    if isinstance(row["belongs_to_collection"], float):
        print(index, row["belongs_to_collection"])



19730 0.065736
29503 1.931659
35587 2.185485


In [ ]:
# Counting the number of values in the column
print(len(movies['belongs_to_collection']))

# Counting the number of values in the dataframe
print(len(movies))
# 45466

45466
45466


In [ ]:
# I remove those 3 rows from the whole dataframe since they had changed values and may generate future errors.
movies = movies.drop(movies[movies['belongs_to_collection'].apply(lambda x: isinstance(x, float))].index)

In [ ]:
# Counting the number of values in the dataframe to verify the droped ones.
print(len(movies))
# 45463


45463


In [ ]:
# Define the function to extract franchise names
# This function assumes that float values have already been removed, so it is not necessary to check if the value is float or nan.
def extract_name(collection):
    if isinstance(collection, dict):
        return collection.get('name')
    return None

# def extract_name(collection):
#     if isinstance(collection, dict):
#         return collection['name']  # If the collection is a dictionary, return the value associated with the key 'name'
#     elif isinstance(collection, float) and math.isnan(collection):
#         return None  # If the collection is a NaN (Not a Number) float value, return None
#     else:
#         return None  # For any other case, return None


# Apply the function to the column to extract the names and replace it
movies['belongs_to_collection'] = movies['belongs_to_collection'].apply(extract_name)

print(movies['belongs_to_collection'])


0                  Toy Story Collection
1                                  None
2             Grumpy Old Men Collection
3                                  None
4        Father of the Bride Collection
                      ...              
45461                              None
45462                              None
45463                              None
45464                              None
45465                              None
Name: belongs_to_collection, Length: 45463, dtype: object


In [ ]:
# Veryfing count of non-null records
non_null_count = len(movies['belongs_to_collection']) - movies['belongs_to_collection'].isnull().sum()
print(non_null_count)


4491


## **genres** column

In [ ]:
# Checks for null values
movies['genres'].isnull().sum()

0

In [ ]:
# Verifying the data of the elements of "genres". They are a list of dictionaries in string format.
print(movies['genres'][0])
type(movies['genres'][0])

[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]


str

In [ ]:
# Using the created function to convert the 'genres' column into a list of dictionaries
movies['genres'] = movies['genres'].apply(convert_to_dict)

In [ ]:
# Creating a function that iterates over each dictionary in the "genres" list and returns a list of genres for each movie, intended for the ML model.
def get_genres(movie_genres):
    genres_list = []
    for genre in movie_genres:
        genres_list.append(genre['name'])
    return genres_list

# Creating a new column in the "movies" DataFrame by applying the "get_genres" function to each record in the "genres" column:
movies['genres']= movies['genres'].apply(get_genres)

In [ ]:
movies.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,Toy Story Collection,30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,None,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,Grumpy Old Men Collection,0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


## **production_companies** column

In [ ]:
# Checks for null values
movies['production_companies'].isnull().sum()

3

In [ ]:
# Verifying the data of the elements of "production_companies". They are a list of dictionaries in string format.
print(movies['production_companies'][0])
type(movies['production_companies'][0])

[{'name': 'Pixar Animation Studios', 'id': 3}]


str

In [ ]:
# Applying the convert_to_dict function to the 'production_companies' column
movies['production_companies'] = movies['production_companies'].apply(convert_to_dict)

In [ ]:
# Creating a function that iterates over each dictionary in the "production_companies" list and returns a list of company names.
def get_companies(compania):
  companies=[]
  if compania != None and type(compania) != bool:
    try:
      for dicc in compania:
        companies.append(dicc["name"])
      return companies
    except (ValueError, SyntaxError):
      return None
  else:
    return None

# Applying the "get_companies" function to each record in the "production_companies" column and creating a new column "production_companies":
movies['production_companies'] = movies['production_companies'].apply(get_companies)

In [ ]:
print(movies['production_companies'])
print(len(movies['production_companies']))

0                                [Pixar Animation Studios]
1        [TriStar Pictures, Teitler Film, Interscope Co...
2                           [Warner Bros., Lancaster Gate]
3                 [Twentieth Century Fox Film Corporation]
4             [Sandollar Productions, Touchstone Pictures]
                               ...                        
45461                                                   []
45462                                        [Sine Olivia]
45463                            [American World Pictures]
45464                                          [Yermoliev]
45465                                                   []
Name: production_companies, Length: 45463, dtype: object
45463


In [ ]:
# Checks for null values again
movies['production_companies'].isnull().sum()
# Tengo 3 nulos

3

In [ ]:
# Counting the empty lists in the Series, considering None values
empty_lists_count = movies['production_companies'].apply(lambda x: len(x) == 0 if x is not None else False).sum()
print(empty_lists_count)


11875


In [ ]:
# Finding the indices of null values in the 'production_companies' column (i think they do not affect)
null_indices = movies[movies['production_companies'].isnull()].index
print(null_indices)

Int64Index([19729, 29502, 35586], dtype='int64')


## **production_countries** column

In [ ]:
# Checks for null values
movies['production_countries'].isnull().sum()

3

In [ ]:
# Verifying the data of the elements of "production_countries". They are a list of dictionaries in string format.
print(movies['production_countries'][0])
type(movies['production_countries'][0])


[{'iso_3166_1': 'US', 'name': 'United States of America'}]


str

In [ ]:
# Applying the convert_to_dict function to the 'production_countries' column
movies['production_countries'] = movies['production_countries'].apply(convert_to_dict)

In [ ]:
movies['production_countries'].head()

0    [{'iso_3166_1': 'US', 'name': 'United States o...
1    [{'iso_3166_1': 'US', 'name': 'United States o...
2    [{'iso_3166_1': 'US', 'name': 'United States o...
3    [{'iso_3166_1': 'US', 'name': 'United States o...
4    [{'iso_3166_1': 'US', 'name': 'United States o...
Name: production_countries, dtype: object

In [ ]:
# Creating a function that iterates over each dictionary in the "production_countries" list and returns a list of countries names.
def get_countries(countrie):
  countries=[]
  if countrie!= None and type(countrie)!=float:
    try:
      for dicc in countrie:
        countries.append(dicc["name"])
      return countries
    except (ValueError, SyntaxError):
      return None
  else:
      return None

# Modifying the 'production_countries' column to contain only a list of countries:
movies["production_countries"]=movies["production_countries"].apply(get_countries)

In [ ]:
print(movies['production_countries'].head(3))
print(len(movies['production_countries']))

0    [United States of America]
1    [United States of America]
2    [United States of America]
Name: production_countries, dtype: object
45463


# Merge of DataFrames: credits and movies
* The merge will combine the information from the "credits" DataFrame and the "movies" DataFrame into a single DataFrame.
* We will use the 'id' column as the key to merge the DataFrames.
* The inner join is performed on the 'id' column. This means that only the rows where the 'id' values are present in both the "movies" and "credits" DataFrames will be included in the resulting merged DataFrame, "df_movies".


In [ ]:
# Counting the duplicated values in the 'id' column of the 'credits' DataFrame before merging
print(credits['id'].duplicated().sum())
#44

# Counting the duplicated values in the 'id' column of the 'movies' DataFrame before merging
print(movies['id'].duplicated().sum())
#30

# Removing duplicates from the 'movies' and 'credits' DataFrames before merging
movies.drop_duplicates(subset='id', inplace=True)
credits.drop_duplicates(subset='id', inplace=True)

# Converting the 'id' column in the 'credits' DataFrame to string
credits['id'] = credits['id'].astype(str)

# Converting the 'id' column in the 'movies' DataFrame to string
movies['id'] = movies['id'].astype(str)

# Merging the "movies" and "credits" DataFrames based on the 'id' column
df_movies = pd.merge(movies, credits, on='id')

44
30


In [ ]:
print(df_movies.head(4))

   adult      belongs_to_collection    budget                        genres  \
0  False       Toy Story Collection  30000000   [Animation, Comedy, Family]   
1  False                       None  65000000  [Adventure, Fantasy, Family]   
2  False  Grumpy Old Men Collection         0             [Romance, Comedy]   
3  False                       None  16000000      [Comedy, Drama, Romance]   

                               homepage     id    imdb_id original_language  \
0  http://toystory.disney.com/toy-story    862  tt0114709                en   
1                                   NaN   8844  tt0113497                en   
2                                   NaN  15602  tt0113228                en   
3                                   NaN  31357  tt0114885                en   

      original_title                                           overview  ...  \
0          Toy Story  Led by Woody, Andy's toys live happily in his ...  ...   
1            Jumanji  When siblings Judy and Pet

In [ ]:
# Elimino columnas que no usaré
columns_to_drop = ['video','imdb_id','adult','original_title', 'poster_path', 'homepage']
df_movies = df_movies.drop(columns_to_drop, axis = 1)

In [ ]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45432 entries, 0 to 45431
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4488 non-null   object 
 1   budget                 45432 non-null  object 
 2   genres                 45432 non-null  object 
 3   id                     45432 non-null  object 
 4   original_language      45421 non-null  object 
 5   overview               44478 non-null  object 
 6   popularity             45429 non-null  object 
 7   production_companies   45429 non-null  object 
 8   production_countries   45429 non-null  object 
 9   release_date           45345 non-null  object 
 10  revenue                45429 non-null  float64
 11  runtime                45172 non-null  float64
 12  spoken_languages       45429 non-null  object 
 13  status                 45348 non-null  object 
 14  tagline                20401 non-null  object 
 15  ti

In [ ]:
# Verifying the duplicated values in the 'id' column of the merged DataFrame, df_movies
print(df_movies['id'].duplicated().sum())


0


# **ETL**:
## Transformations: For this MVP you don't need perfection, you need speed! ⏩ You will do these, and only these, transformations to the data




* Some fields, such as `belongs_to_collection`,  `production_companies`, and others (see data dictionary), are nested. This means that they either have a dictionary or a list as values in each row. You will need to unnest them in order to join them back to the dataset or find a way to access that data without unnesting.

* Null values in the `revenue` and `budget` fields should be filled with the number 0.

* Null values in the`release_date` field should be removed.

* If there are dates, they should be in the format YYYY-mm-dd. Additionally, you should create the `release_year` column to extract the year from the release date.

* Create a column for return on investment called  `return` with the `revenue` and `budget` fields, dividing the revenue by the budget. If there are no available data to calculate it, the value should be 0.

* Remove the columns that will not be used: `video,imdb_id,adult,original_title,poster_path and homepage.`



## ***Revenue***

In [ ]:
# Searching for null or empty values in the 'revenue' column
null_values = df_movies['revenue'].isnull()
empty_values = df_movies['revenue'] == ''

# Printing the results
print("Null values in revenue:")
print(null_values.sum())

print("Empty values in revenue:")
print(empty_values.sum())




Null values in revenue:
3
Empty values in revenue:
0


In [ ]:
# Replacing null values with 0 in the 'revenue' column
df_movies['revenue'] = df_movies['revenue'].fillna(0)

## ***Budget***

In [ ]:
# Converting the 'budget' column to float
df_movies['budget'] = df_movies['budget'].astype(float)

In [ ]:
# Searching for null or empty values in the 'budget' column
null_values = df_movies['budget'].isnull()
empty_values = df_movies['budget'] == ''

print("Null values in budget:")
print(null_values.sum())

print("Empty values in budget:")
print(empty_values.sum())

Null values in budget:
0
Empty values in budget:
0


In [ ]:
# Reemplazar los valores nulos por 0
df_movies['budget'] = df_movies['budget'].fillna(0)

## ***Release_date***

In [ ]:
# Counting the null values in the 'release_date' column
null_values_count = df_movies['release_date'].isnull().sum()

# Printing the result
print("Null values in release_date:", null_values_count)
# 87


Null values in release_date: 87


In [ ]:
# Dropping the null values from the 'release_date' column
df_movies.dropna(subset=['release_date'], inplace=True)

In [ ]:
# Converting the DataSeries to date format
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], format='%Y-%m-%d', errors='coerce')
# errors='coerce': This parameter indicates how to handle errors during the conversion.

In [ ]:
# Creating a new column for the year extracted from the 'release_date'
df_movies["release_year"]= df_movies["release_date"].dt.year

## ***Return***

Create the column with the return on investment, called return with the revenue and budget fields, dividing these last two revenue / budget, when there is no data available to calculate it, it must take the value 0.

In [ ]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45345 entries, 0 to 45431
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  4485 non-null   object        
 1   budget                 45345 non-null  float64       
 2   genres                 45345 non-null  object        
 3   id                     45345 non-null  object        
 4   original_language      45334 non-null  object        
 5   overview               44404 non-null  object        
 6   popularity             45345 non-null  object        
 7   production_companies   45345 non-null  object        
 8   production_countries   45345 non-null  object        
 9   release_date           45345 non-null  datetime64[ns]
 10  revenue                45345 non-null  float64       
 11  runtime                45099 non-null  float64       
 12  spoken_languages       45345 non-null  object        
 13  s

In [ ]:
# Calculating the return on investment and assigning 0 when the budget is 0
df_movies['return'] = df_movies.apply(lambda row: row['revenue'] / row['budget'] if row['revenue'] and row['budget'] else 0, axis=1)
df_movies.head()


,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,director,actors,release_year,return
0,Toy Story Collection,30000000.0,"[Animation, Comedy, Family]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,[John Lasseter],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...",1995,12.451801
1,None,65000000.0,"[Adventure, Fantasy, Family]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,[Joe Johnston],"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...",1995,4.043035
2,Grumpy Old Men Collection,0.0,"[Romance, Comedy]",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,[Howard Deutch],"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...",1995,0.000000
3,None,16000000.0,"[Comedy, Drama, Romance]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,[Forest Whitaker],"[Whitney Houston, Angela Bassett, Loretta Devi...",1995,5.090760
4,Father of the Bride Collection,0.0,[Comedy],11862,en,Just when George Banks has recovered from his ...,8.387519,"[Sandollar Productions, Touchstone Pictures]",[United States of America],1995-02-10,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,[Charles Shyer],"[Steve Martin, Diane Keaton, Martin Short, Kim...",1995,0.000000


In [ ]:
# Counting the null values in the 'return' column
null_values_count = df_movies['return'].isnull().sum()

# Printing the result
print("Null values in return:", null_values_count)


Null values in return: 0


In [ ]:
df_movies['return'][0]

12.4518011

In [ ]:
df_movies['return'].head(10)

0    12.451801
1     4.043035
2     0.000000
3     5.090760
4     0.000000
5     3.123947
6     0.000000
7     0.000000
8     1.838576
9     6.072311
Name: return, dtype: float64

# Rearrange columns order

In [ ]:
# Define the desired column order
desired_column_order = ['id', 'title', 'belongs_to_collection', 'genres',  'budget', 'original_language', 'overview', 'popularity', 'production_companies', 'production_countries', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'vote_average', 'vote_count', 'director', 'actors', 'release_year', 'return']

# Reorder the columns in the DataFrame
df_movies = df_movies[desired_column_order]

# Exporting CSV

In [ ]:
# # Define the file path to save the DataFrame as a CSV file
savefile_path = '/dataset/df_movies.csv'

# # Save the DataFrame as a CSV file without including the index
df_movies.to_csv(savefile_path, index=False)


# Drive
# savefile_path = '/content/drive/MyDrive/Proyectos Henry/ML_Ops_Project/df_movies.csv'
# df_movies.to_csv(savefile_path, index=False)

In [ ]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45345 entries, 0 to 45431
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   id                     45345 non-null  object        
 1   title                  45345 non-null  object        
 2   belongs_to_collection  4485 non-null   object        
 3   genres                 45345 non-null  object        
 4   budget                 45345 non-null  float64       
 5   original_language      45334 non-null  object        
 6   overview               44404 non-null  object        
 7   popularity             45345 non-null  object        
 8   production_companies   45345 non-null  object        
 9   production_countries   45345 non-null  object        
 10  release_date           45345 non-null  datetime64[ns]
 11  revenue                45345 non-null  float64       
 12  runtime                45099 non-null  float64       
 13  s